# Building the dataset
In this notebook I build the dataset I will use for the neural network.<br>
The steps to follow are:<br>
1. Extract the GWAS catalog `rs` ids
2. Build the corresponding sequences to those variants with the help of `bcftools`
3. Build random sequences from the reference genome (following the same method as the previous step)
4. Tokenize the dataset with `DNABERT`'s tokenizer
5. Build the `PyTorch` dataloaders

In [1]:
import pandas as pd
import os
from Bio import SeqIO
from Bio.Seq import MutableSeq, Seq
import subprocess

In [2]:
databases_path = "/mnt/sda1/Databases/"
gwas_catalog_path = os.path.join(databases_path, "GWAS_Catalog_DATA/gwas_catalog_v1.0.2-associations_e110_r2023-09-25.tsv")
ensembl_path = os.path.join(databases_path, "Ensembl/Variation/110/")
chromosomes_path = os.path.join(ensembl_path, "chromosomes_data/")

## Extract the GWAS Catalog `rs` ids

In [3]:
gwas_catalog = pd.read_csv(gwas_catalog_path, delimiter="\t", dtype=str)
gwas_catalog.head(n=3)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,27.0,NaN,7.4,[6.03-8.77] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array
1,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,5.522878745280337,NaN,3.5,[1.93-5.07] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array
2,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,7.0,NaN,3.6,[2.23-4.97] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array


In [4]:
# Extract all the variants containing the `rs` substring and creating a new Series object to not affect the original dataframe
gwas_catalog_rs_filtered = gwas_catalog[gwas_catalog["SNPS"].str.contains("rs", case=False, na=False)].copy(deep=True)
gwas_catalog_rs_filtered.head(n=3)

,DATE ADDED TO CATALOG,PUBMEDID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,PVALUE_MLOG,P-VALUE (TEXT),OR or BETA,95% CI (TEXT),PLATFORM [SNPS PASSING QC],CNV,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY
0,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,27.0,NaN,7.4,[6.03-8.77] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array
1,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,5.522878745280337,NaN,3.5,[1.93-5.07] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array
2,2008-07-01,18391951,Gudbjartsson DF,2008-04-06,Nat Genet,www.ncbi.nlm.nih.gov/pubmed/18391951,Many sequence variants affecting diversity of ...,Height,"30,968 European ancestry individuals","8,541 European ancestry individuals",...,7.0,NaN,3.6,[2.23-4.97] % s.d. increase,"Affymetrix, Illumina [up to 304226]",N,body height,http://www.ebi.ac.uk/efo/EFO_0004339,GCST000175,Genome-wide genotyping array


In [6]:
rs_ids = gwas_catalog_rs_filtered.SNPS
rs_ids.head()

0    rs6763931
1    rs1474563
2    rs4794665
3    rs6899976
4     rs749052
Name: SNPS, dtype: object

Let's start by extracting the variants from chromosome 1. The `gwas_catalog_rs_filtered` dataframe has mixed data types in some columns, the `CHR_ID` is one of them. We will change its data type into `str`.

In [6]:
# Use a dictinary to convert specific columns 
#convert_dict = {"CHR_ID": str}

# Appy the data type transformation
#gwas_catalog_rs_filtered = gwas_catalog_rs_filtered.astype(convert_dict)

In [6]:
gwas_catalog_rs_filtered[gwas_catalog_rs_filtered["CHR_ID"]=="1"].shape

(45989, 38)

In [11]:
chr1_ids = gwas_catalog_rs_filtered[gwas_catalog_rs_filtered["CHR_ID"]=="1"].SNPS.to_list()
print(len(chr1_ids), chr1_ids[:5])

45989 ['rs11205277', 'rs2274432', 'rs678962', 'rs646776', 'rs34372695']


### Import Chromosome 1 data

In [7]:
chr1_data = pd.read_csv(os.path.join(chromosomes_path, "chr1_data.tsv"), delimiter="\t", dtype=str)
chr1_data.rename(columns={'#[1]CHROM':'chr', '[2]POS':'pos', '[3]REF':'ref', '[4]ALT':'alt', '[5]TSA':'tsa', '[6]ID':'id'}, inplace=True)
chr1_data.head()

,chr,pos,ref,alt,tsa,id
0,1,10001,T,A,SNV,rs1570391677
1,1,10002,A,C,SNV,rs1570391692
2,1,10003,A,C,SNV,rs1570391694
3,1,10008,A,G,SNV,rs1570391698
4,1,10009,A,G,SNV,rs1570391702


In [9]:
chr1_data.head()

,chr,pos,ref,alt,tsa,id
0,1,10001,T,A,SNV,rs1570391677
1,1,10002,A,C,SNV,rs1570391692
2,1,10003,A,C,SNV,rs1570391694
3,1,10008,A,G,SNV,rs1570391698
4,1,10009,A,G,SNV,rs1570391702


The `chr1_data` has mixed data types in the `#[1]CHROM` column too. We have to change all the values to strings. **Fixed by adding the `dtype` argument and setting it to `str`**.

In [9]:
#chr_convert_dict = {"#[1]CHROM": str}
#chr1_data = chr1_data.astype(chr_convert_dict)
#chr1_data["#[1]CHROM"].value_counts()

#[1]CHROM
1    54950386
Name: count, dtype: int64

Now I extract the variants that have been associated from all the registered ones in Ensembl

In [12]:
chr1_gwas_variants = chr1_data[chr1_data["id"].isin(chr1_ids)].copy(deep=True)
chr1_gwas_variants.head()

,chr,pos,ref,alt,tsa,id
100427,1,832873,A,"C,T",SNV,rs2977608
103612,1,845017,C,T,SNV,rs141175086
136253,1,946653,G,"A,C",SNV,rs2272756
139785,1,955679,C,"A,T",SNV,rs13303065
141251,1,959139,G,A,SNV,rs115438739


In [13]:
print(chr1_gwas_variants.shape)

(22453, 6)


In [16]:
gwas_catalog_all_rs = gwas_catalog_rs_filtered["SNPS"].to_list()
gwas_catalog_all_rs[:5]

['rs6763931', 'rs1474563', 'rs4794665', 'rs6899976', 'rs749052']

In [14]:
gwas_catalog_rs_filtered["SNPS"].value_counts()

SNPS
rs704          1873
rs1260326      1045
rs964184        974
rs1354034       631
rs7212510       567
               ... 
rs61879614        1
rs34319244        1
rs113114656       1
rs67213764        1
rs2483289         1
Name: count, Length: 265265, dtype: int64

**The number of rs ids from the GWAS catalog and the variants extracted from the Ensembl chromosome 1 doesn't match because I have repeated IDs in the GWAS catalog**.<br>
This means that some variants in the GWAS catalog are associated with more than one phenotype.

In [13]:
# Filter the gwas_catalog_rs_filtered variable by only returning the lines in which the rs1260326 variant appears, and check the registers' phenotype
gwas_catalog_rs_filtered[gwas_catalog_rs_filtered["SNPS"]=="rs1260326"]["DISEASE/TRAIT"].value_counts()

DISEASE/TRAIT
Metabolite levels                                                      29
Serum metabolite levels                                                23
Serum metabolite levels (CMS)                                          19
Triglyceride levels                                                    18
Total cholesterol levels                                               18
                                                                       ..
Concentration of HDL particles (UKB data field 23430)                   1
Total concentration of lipoprotein particles (UKB data field 23427)     1
Total lipids in lipoprotein particles (UKB data field 23423)            1
Total lipids in VLDL (UKB data field 23424)                             1
Weight                                                                  1
Name: count, Length: 697, dtype: int64

In [17]:
# Extract the variants from the ensembl database with the id's extracted from the GWAS catalog
chr1_gwas_variants = chr1_data[chr1_data["id"].isin(gwas_catalog_all_rs)].copy(deep=True)
chr1_gwas_variants.head()

,chr,pos,ref,alt,tsa,id
100427,1,832873,A,"C,T",SNV,rs2977608
103612,1,845017,C,T,SNV,rs141175086
136253,1,946653,G,"A,C",SNV,rs2272756
139785,1,955679,C,"A,T",SNV,rs13303065
141251,1,959139,G,A,SNV,rs115438739


In [21]:
chr1_gwas_variants["tsa"].value_counts()

tsa
SNV             21593
indel             708
insertion          84
deletion           82
substitution        1
Name: count, dtype: int64

In [14]:
gwas_catalog_rs_filtered["SNPS"].value_counts()

SNPS
rs704          1873
rs1260326      1045
rs964184        974
rs1354034       631
rs7212510       567
               ... 
rs61879614        1
rs34319244        1
rs113114656       1
rs67213764        1
rs2483289         1
Name: count, Length: 265265, dtype: int64

**The code for generating the sequences is limited to SNVs, therefore we will work only with them for now.**

In [22]:
# Create a new df with only the SNV variants from the filtered ensembl variants
chr1_gwas_snps = chr1_gwas_variants[chr1_gwas_variants["tsa"]=="SNV"].copy(deep=True)
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id
100427,1,832873,A,"C,T",SNV,rs2977608
103612,1,845017,C,T,SNV,rs141175086
136253,1,946653,G,"A,C",SNV,rs2272756
139785,1,955679,C,"A,T",SNV,rs13303065
141251,1,959139,G,A,SNV,rs115438739


## Generate the sequences
I will use the `generate_bed` and the `generate_sequences` functions from the `process_data.py` file. <br>
Instead of importing them, I will paste the codes and make adjustments to make them work with multiallelic variants.

In [ ]:
# generate_bed function
def generate_bed(data_path: str, chromosome: str, res_path: str, gen_bed:bool = False):
    '''
    Function to process the data from the VCF files from Ensembl Variation.
    Limited to single alternate alleles and SNPs. Check the `data_processing.ipynb` file.

    Variables:
    ----------
    data_path: path where the tsv files extracted by bcftools from the original vcf files are stored

    res_path: path where the produced files will be saved

    chromosome: number of chromosome to process

    gen_bed: A boolean value to indicate if the `bed` column will be exported into a csv file
    '''
    
    #data_path = data_path
    #res_path = res_path
    #chromosome = chromosome
    #chr_n_data = pd.read_csv(data_path+'chr{}_data.tsv'.format(chromosome), sep = '\t')
    chr_n_data.rename(columns={'#[1]CHROM':'chr', '[2]POS':'pos', '[3]REF':'ref', '[4]ALT':'alt', '[5]TSA':'tsa', '[6]ID':'id'}, inplace=True)
    
    # Filter the variants that are snps and that have only one alternate allele
    #chr_n_snps = chr_n_data.loc[(chr_n_data['tsa']=='SNV') & (chr_n_data['alt'].str.len() == 1)]

    chr1_gwas_snps['start'] = chr_n_snps['pos'].astype(int) - 64
    chr_n_snps['end'] = chr_n_snps['pos'].astype(int) + 63
    chr_n_snps['bed'] = chr_n_snps['chr'].astype(str) + ':' + chr_n_snps['start'].astype(str) + '-' + chr_n_snps['end'].astype(str)
    if gen_bed == True: chr_n_snps['bed'].to_csv('{}/bed_chr{}'.format(res_path, chromosome), sep='\t', index=False, header=False)
    return chr_n_snps

In [17]:
# First, we change the name of the columns to access to them easier
chr1_gwas_snps.rename(columns={'#[1]CHROM':'chr', '[2]POS':'pos', '[3]REF':'ref', '[4]ALT':'alt', '[5]TSA':'tsa', '[6]ID':'id'}, inplace=True)
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id
100427,1,832873,A,"C,T",SNV,rs2977608
103612,1,845017,C,T,SNV,rs141175086
136253,1,946653,G,"A,C",SNV,rs2272756
139785,1,955679,C,"A,T",SNV,rs13303065
141251,1,959139,G,A,SNV,rs115438739


In [18]:
chr1_gwas_snps.shape

(21593, 6)

In [19]:
chr1_gwas_snps['start'] = chr1_gwas_snps['pos'].astype(int) - 63
chr1_gwas_snps['end'] = chr1_gwas_snps['pos'].astype(int) + 64
chr1_gwas_snps['bed'] = chr1_gwas_snps['chr'].astype(str) + ':' + chr1_gwas_snps['start'].astype(str) + '-' + chr1_gwas_snps['end'].astype(str)
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed
100427,1,832873,A,"C,T",SNV,rs2977608,832810,832937,1:832810-832937
103612,1,845017,C,T,SNV,rs141175086,844954,845081,1:844954-845081
136253,1,946653,G,"A,C",SNV,rs2272756,946590,946717,1:946590-946717
139785,1,955679,C,"A,T",SNV,rs13303065,955616,955743,1:955616-955743
141251,1,959139,G,A,SNV,rs115438739,959076,959203,1:959076-959203


In [20]:
chr1_gwas_snps["alt_list"] = chr1_gwas_snps["alt"].str.split(pat=",")
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed,alt_list
100427,1,832873,A,"C,T",SNV,rs2977608,832810,832937,1:832810-832937,"[C, T]"
103612,1,845017,C,T,SNV,rs141175086,844954,845081,1:844954-845081,[T]
136253,1,946653,G,"A,C",SNV,rs2272756,946590,946717,1:946590-946717,"[A, C]"
139785,1,955679,C,"A,T",SNV,rs13303065,955616,955743,1:955616-955743,"[A, T]"
141251,1,959139,G,A,SNV,rs115438739,959076,959203,1:959076-959203,[A]


In [21]:
chr1_gwas_snps["alt_list"].iloc[2]

['A', 'C']

### Save the bed-like strings into a text file to read them with `samtools`

**To generate the sequences I have to save the `bed` column into a text file so it can be read by `samtools`**.

In [22]:
# Path where the bed columns will be saved
gwas_associated_bed_path = os.path.join(databases_path, "Ensembl/Variation/110/gwas_associated_sequences/beds")
chromosome = "1"
# The following line was commented to not create new files each time this cell is executed
chr1_gwas_snps['bed'].to_csv('{}/bed_chr{}'.format(gwas_associated_bed_path, chromosome), sep='\t', index=False, header=False)

In [10]:
# Path where the reference subsequences will be saved
gwas_associated_seq_path = os.path.join(databases_path, "Ensembl/Variation/110/gwas_associated_sequences/ref_sequences/")
print(gwas_associated_seq_path)

/mnt/sda1/Databases/Ensembl/Variation/110/gwas_associated_sequences/ref_sequences/


In [ ]:
# build_sequences function
def generate_sequences(ref_gen_path:str, res_path: str, chromosome: str, variant_df: pd.DataFrame, generate_fasta: bool = False):
    if generate_fasta == True:
        # This code has to be run directly in the terminal after adding the samtools path to the PATH variable
        exit_code = subprocess.Popen("samtools faidx {} -r {}/bed_chr{} -o {}/seq_vcf_chr{}".format(ref_gen_path, res_path, chromosome, res_path, chromosome), 
                                    shell=True, stdout=subprocess.PIPE).stdout.read()
        if str(exit_code,'utf-8')!='':
            print(str(exit_code,'utf-8'))
            exit(1)
    
    records = list(SeqIO.parse(res_path+'seq_vcf_chr{}'.format(chromosome), "fasta"))
    ref_seqs = [sequence[1].seq for sequence in enumerate(records)]

    alt_alleles = list(variant_df['alt'])
    alt_seqs = []
    i = 0
    while i < len(ref_seqs):
        mutable = MutableSeq(ref_seqs[i])
        mutable[64] = alt_alleles[i]
        alt_seqs.append(str(mutable))
        i+=1
    
    ref_seqs = list(map(str, ref_seqs)) # Turn Seq objects into normal Python Strings and save them in a list

    chr_n_df = pd.DataFrame(data = variant_df, copy=True)
    chr_n_df['ref_seq'] = ref_seqs
    chr_n_df['alt_seq'] = alt_seqs

    return chr_n_df

In [9]:
# Path where the reference genome I use is stored
ref_genome_path = os.path.join(databases_path,"Reference_Genome/GRCh38p14/Ensembl/Homo_sapiens_GRCh38_dna_primary_assembly.fa")
print(ref_genome_path)

/mnt/sda1/Databases/Reference_Genome/GRCh38p14/Ensembl/Homo_sapiens_GRCh38_dna_primary_assembly.fa


**Generate sequences from the reference genome**:

In [25]:
exit_code = subprocess.Popen("samtools faidx {} -r {}/bed_chr{} -o {}/ref_seq_chr{}".format(ref_genome_path, gwas_associated_bed_path, chromosome, gwas_associated_seq_path, chromosome), 
                                    shell=True, stdout=subprocess.PIPE).stdout.read()

**Load generated sequences to modify them**:

In [45]:
records = list(SeqIO.parse(gwas_associated_seq_path+'ref_seq_chr{}'.format(chromosome), "fasta"))
ref_seqs = [str(sequence[1].seq) for sequence in enumerate(records)]
print(len(ref_seqs),ref_seqs[:5])

21593 ['TGTAGTGCAGTGGCGTGATCTTGGCTCACTGCAGCCTCCACCTTAGAGCAATCCTCTTGCCTCATCCTCCCGGGTAGTTGGGACTACATGTGCATGCCACATGCCTGGCTAATTTTTGTATTTTTAGT', 'CCTTTTATTCTATTTTAATTAATTTAAATTACAATAAGTATTATCACTAAGTTTATGCAAACACGTGAGGGAATGCTGATTTAGAGCTGGTGGCGTAAGGTCACAGAGCTCCCACGAATCTCACATGG', 'TGTGCACAGCTGGCCCAGGTCCTGTGCAAAACCACGCGTGGTGGCCACGGGGATACCCCAGGAGGGGACATGGATCCCATCTCAGGGCTCAAGTGCATAGCTGTTGCAGCTGGGATGGCAGAGGCAGA', 'GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGCAGCTGCACACGCCACACCTTCTGAGACTCTGAGCAGACAACTTCCCTTTTCAGGAAGGAAAGAAGGTGGGGCCGCTCCAACT', 'AGGCGCCTGCGGGTCACGCAGGAGTCACAGCTGCCCGCACGCCCAGCTCGCCCCAGCCCCGCTGAGAGGAGCAAGAAAAGCCCCCTTGGATACAGACACCCACCGGGAGGCCAAATCGGCCCTCGGAC']


## Include the reference sequences into the chromosome df

In [47]:
chr1_gwas_snps["ref_seq"] = ref_seqs
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed,alt_list,ref_seq
100427,1,832873,A,"C,T",SNV,rs2977608,832810,832937,1:832810-832937,"[C, T]",TGTAGTGCAGTGGCGTGATCTTGGCTCACTGCAGCCTCCACCTTAG...
103612,1,845017,C,T,SNV,rs141175086,844954,845081,1:844954-845081,[T],CCTTTTATTCTATTTTAATTAATTTAAATTACAATAAGTATTATCA...
136253,1,946653,G,"A,C",SNV,rs2272756,946590,946717,1:946590-946717,"[A, C]",TGTGCACAGCTGGCCCAGGTCCTGTGCAAAACCACGCGTGGTGGCC...
139785,1,955679,C,"A,T",SNV,rs13303065,955616,955743,1:955616-955743,"[A, T]",GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGC...
141251,1,959139,G,A,SNV,rs115438739,959076,959203,1:959076-959203,[A],AGGCGCCTGCGGGTCACGCAGGAGTCACAGCTGCCCGCACGCCCAG...


In [73]:
# Generate the subsequences for variants, including the multiallelic ones
alt_seq = []
for idx, variant in enumerate(chr1_gwas_snps['alt_list']):
    tmp_alt_seq = [] # Clear the contents of this list each time the for loop goes to a new register
    for allele in variant:
        tmp_seq = MutableSeq(chr1_gwas_snps['ref_seq'].iloc[idx])
        tmp_seq[63] = allele
        tmp_seq = str(tmp_seq)
        tmp_alt_seq.append(tmp_seq)
    alt_seq.append(tmp_alt_seq)

alt_seq

[['...'],
 ...]

In [74]:
len(alt_seq)

21593

In [75]:
chr1_gwas_snps['alt_seq'] = alt_seq
chr1_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed,alt_list,ref_seq,alt_seq
100427,1,832873,A,"C,T",SNV,rs2977608,832810,832937,1:832810-832937,"[C, T]",TGTAGTGCAGTGGCGTGATCTTGGCTCACTGCAGCCTCCACCTTAG...,[TGTAGTGCAGTGGCGTGATCTTGGCTCACTGCAGCCTCCACCTTA...
103612,1,845017,C,T,SNV,rs141175086,844954,845081,1:844954-845081,[T],CCTTTTATTCTATTTTAATTAATTTAAATTACAATAAGTATTATCA...,[CCTTTTATTCTATTTTAATTAATTTAAATTACAATAAGTATTATC...
136253,1,946653,G,"A,C",SNV,rs2272756,946590,946717,1:946590-946717,"[A, C]",TGTGCACAGCTGGCCCAGGTCCTGTGCAAAACCACGCGTGGTGGCC...,[TGTGCACAGCTGGCCCAGGTCCTGTGCAAAACCACGCGTGGTGGC...
139785,1,955679,C,"A,T",SNV,rs13303065,955616,955743,1:955616-955743,"[A, T]",GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGC...,[GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGG...
141251,1,959139,G,A,SNV,rs115438739,959076,959203,1:959076-959203,[A],AGGCGCCTGCGGGTCACGCAGGAGTCACAGCTGCCCGCACGCCCAG...,[AGGCGCCTGCGGGTCACGCAGGAGTCACAGCTGCCCGCACGCCCA...


In [82]:
dummy = chr1_gwas_snps.iloc[3]
print('', dummy['ref_seq'], '\n', dummy['alt_seq'][0], '\n', dummy['alt_seq'][1])

 GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGCAGCTGCACACGCCACACCTTCTGAGACTCTGAGCAGACAACTTCCCTTTTCAGGAAGGAAAGAAGGTGGGGCCGCTCCAACT 
 GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGCAGCTGCACACGCCACACATTCTGAGACTCTGAGCAGACAACTTCCCTTTTCAGGAAGGAAAGAAGGTGGGGCCGCTCCAACT 
 GAAATAGAGCAAGAACCCTAACCTGGGAAAATCTGAGAGCAAAGGCAGCTGCACACGCCACACTTTCTGAGACTCTGAGCAGACAACTTCCCTTTTCAGGAAGGAAAGAAGGTGGGGCCGCTCCAACT


In [83]:
# Save the dataframe for future uses
chr1_gwas_snps.to_csv('/mnt/sda1/Databases/Ensembl/Variation/110/chromosome_datasets/chr1_gwas_dataset.csv')

# Replicate with chromosome 2

## Load chromosome 2 data and GWAS Catalog
The first step is to import the chromosome 2 data (which was extracted from Ensembl Variation 110), and the GWAS Catalog version compatible with Ensembl Build 110.

In [5]:
# Load chromosome 2 data
chr2_data = pd.read_csv(os.path.join(chromosomes_path,'chr2_data.tsv'), delimiter='\t', dtype=str)
chr2_data.rename(columns={'#[1]CHROM':'chr', '[2]POS':'pos', '[3]REF':'ref', '[4]ALT':'alt', '[5]TSA':'tsa', '[6]ID':'id'}, inplace=True)
chr2_data.head()

,chr,pos,ref,alt,tsa,id
0,2,10007,C,A,SNV,rs1572047073
1,2,10014,C,CG,insertion,rs1558169263
2,2,10017,CACCC,C,indel,rs1558169385
3,2,10018,A,"AACA,AACG",insertion,rs1558169388
4,2,10019,CC,C,indel,rs1558169386


## Filter the GWAS associated variants in chromosome 2 with 'rs' IDs
Elements available to work with:
* `gwas_catalog_rs_filtered` dataframe: this is another dataframe object created from the original gwas catalog, but without any variant not starting with 'rs'

In [27]:
#gwas_catalog_rs_filtered = gwas_catalog[gwas_catalog["SNPS"].str.contains("rs", case=False, na=False)].copy(deep=True)
gwas_catalog_rs_filtered[gwas_catalog_rs_filtered["CHR_ID"]=="2"].shape

(41804, 38)

In [6]:
# Extract the gwas associated variants in chromosome 2 into a new data frame
gwas_catalog_chr2 = gwas_catalog_rs_filtered[gwas_catalog_rs_filtered["CHR_ID"]=="2"].copy(deep=True)
gwas_catalog_chr2.SNPS.iloc[:5]

4      rs749052
7     rs3791679
16    rs1052483
40    rs6733301
58    rs6710823
Name: SNPS, dtype: object

In [8]:
# Filter the chromosome 2 variants by the ids registered in the gwas catalog and by the type of sequence alteration (SNV)
chr2_gwas_snps = chr2_data[(chr2_data["id"].isin(gwas_catalog_chr2.SNPS)) & (chr2_data["tsa"]=="SNV") ].copy(deep=True)
chr2_gwas_snps.head()

,chr,pos,ref,alt,tsa,id
9074,2,38938,A,C,SNV,rs11542478
14066,2,58639,C,T,SNV,rs62116661
18575,2,76417,T,"A,C,G",SNV,rs300769
18619,2,76530,C,"G,T",SNV,rs300768
21715,2,89910,G,"A,C,T",SNV,rs300789


## Generate sequences

### Generate bed-like column

In [10]:
chr2_gwas_snps['start'] = chr2_gwas_snps['pos'].astype(int) - 63
chr2_gwas_snps['end'] = chr2_gwas_snps['pos'].astype(int) + 64
chr2_gwas_snps['bed'] = chr2_gwas_snps['chr'].astype(str) + ':' + chr2_gwas_snps['start'].astype(str) + '-' + chr2_gwas_snps['end'].astype(str)
chr2_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed
9074,2,38938,A,C,SNV,rs11542478,38875,39002,2:38875-39002
14066,2,58639,C,T,SNV,rs62116661,58576,58703,2:58576-58703
18575,2,76417,T,"A,C,G",SNV,rs300769,76354,76481,2:76354-76481
18619,2,76530,C,"G,T",SNV,rs300768,76467,76594,2:76467-76594
21715,2,89910,G,"A,C,T",SNV,rs300789,89847,89974,2:89847-89974


### Save the bed-like column to parse through the reference genome and generate the sequences that will be modified

In [13]:
# Path where the bed columns will be saved
gwas_associated_bed_path = os.path.join(databases_path, "Ensembl/Variation/110/gwas_associated_sequences/beds")
chromosome = "2"
# The following line was commented to not create new files each time this cell is executed
chr2_gwas_snps['bed'].to_csv('{}/bed_chr{}'.format(gwas_associated_bed_path, chromosome), sep='\t', index=False, header=False)

### Generate the to-be-modified sequences

In [15]:
# Path where the reference genome I use is stored
ref_genome_path = os.path.join(databases_path,"Reference_Genome/GRCh38p14/Ensembl/Homo_sapiens_GRCh38_dna_primary_assembly.fa")
# Path where the reference subsequences will be saved
gwas_associated_seq_path = os.path.join(databases_path, "Ensembl/Variation/110/gwas_associated_sequences/ref_sequences/")
exit_code = subprocess.Popen("samtools faidx {} -r {}/bed_chr{} -o {}/ref_seq_chr{}".format(ref_genome_path, gwas_associated_bed_path, chromosome, gwas_associated_seq_path, chromosome), 
                                    shell=True, stdout=subprocess.PIPE).stdout.read()

### Load the created sequences and add them to the data frame we are working with

In [18]:
records = list(SeqIO.parse(gwas_associated_seq_path+'ref_seq_chr{}'.format(chromosome), "fasta"))
ref_seqs = [str(sequence[1].seq) for sequence in enumerate(records)]
chr2_gwas_snps["ref_seq"] = ref_seqs
chr2_gwas_snps["alt_list"] = chr2_gwas_snps["alt"].str.split(pat=",")
chr2_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed,ref_seq,alt_list
9074,2,38938,A,C,SNV,rs11542478,38875,39002,2:38875-39002,TGCAAATAGTGTATAGAAAAAGCTCTGTTTAGAAACTGCCATAGCA...,[C]
14066,2,58639,C,T,SNV,rs62116661,58576,58703,2:58576-58703,CTCCACAATAGTCAAAATGAAAGAAAAATACCAAGCCTCTCTCAGC...,[T]
18575,2,76417,T,"A,C,G",SNV,rs300769,76354,76481,2:76354-76481,TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTATA...,"[A, C, G]"
18619,2,76530,C,"G,T",SNV,rs300768,76467,76594,2:76467-76594,GGCAAAGGAGGAGCAAAGGCACCTCTTACATGGTGGCAGGCAAGAG...,"[G, T]"
21715,2,89910,G,"A,C,T",SNV,rs300789,89847,89974,2:89847-89974,AGTACAGGACACACCATCGCAATAAATTAAAAAGGCAAATGCAAAT...,"[A, C, T]"


In [19]:
# Generate the subsequences for variants, including the multiallelic ones
alt_seq = []
for idx, variant in enumerate(chr2_gwas_snps['alt_list']):
    tmp_alt_seq = [] # Clear the contents of this list each time the for loop goes to a new register
    for allele in variant:
        tmp_seq = MutableSeq(chr2_gwas_snps['ref_seq'].iloc[idx])
        tmp_seq[63] = allele
        tmp_seq = str(tmp_seq)
        tmp_alt_seq.append(tmp_seq)
    alt_seq.append(tmp_alt_seq)

chr2_gwas_snps["alt_seqs"] = alt_seq
chr2_gwas_snps.head()

,chr,pos,ref,alt,tsa,id,start,end,bed,ref_seq,alt_list,alt_seqs
9074,2,38938,A,C,SNV,rs11542478,38875,39002,2:38875-39002,TGCAAATAGTGTATAGAAAAAGCTCTGTTTAGAAACTGCCATAGCA...,[C],[TGCAAATAGTGTATAGAAAAAGCTCTGTTTAGAAACTGCCATAGC...
14066,2,58639,C,T,SNV,rs62116661,58576,58703,2:58576-58703,CTCCACAATAGTCAAAATGAAAGAAAAATACCAAGCCTCTCTCAGC...,[T],[CTCCACAATAGTCAAAATGAAAGAAAAATACCAAGCCTCTCTCAG...
18575,2,76417,T,"A,C,G",SNV,rs300769,76354,76481,2:76354-76481,TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTATA...,"[A, C, G]",[TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTAT...
18619,2,76530,C,"G,T",SNV,rs300768,76467,76594,2:76467-76594,GGCAAAGGAGGAGCAAAGGCACCTCTTACATGGTGGCAGGCAAGAG...,"[G, T]",[GGCAAAGGAGGAGCAAAGGCACCTCTTACATGGTGGCAGGCAAGA...
21715,2,89910,G,"A,C,T",SNV,rs300789,89847,89974,2:89847-89974,AGTACAGGACACACCATCGCAATAAATTAAAAAGGCAAATGCAAAT...,"[A, C, T]",[AGTACAGGACACACCATCGCAATAAATTAAAAAGGCAAATGCAAA...


In [20]:
chr2_gwas_snps["alt_seqs"].iloc[2]

['TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTATAAAGAAAAAAGAGGTTGAATGGACTCACAATTCCACGTGGCTGGGGAGGCTTCACAATCATGGCAGAAGGCAAAGGAGGAGCA',
 'TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTATAAAGAAAAAAGAGGTTGACTGGACTCACAATTCCACGTGGCTGGGGAGGCTTCACAATCATGGCAGAAGGCAAAGGAGGAGCA',
 'TTCTCACACTGCCAATAAAAACATAGCTAAGACTGGGTAATTTATAAAGAAAAAAGAGGTTGAGTGGACTCACAATTCCACGTGGCTGGGGAGGCTTCACAATCATGGCAGAAGGCAAAGGAGGAGCA']

In [16]:
chr2_data.drop(chr2_gwas_snps.index, inplace=True)
chr2_data.head()

,chr,pos,ref,alt,tsa,id
0,2,10007,C,A,SNV,rs1572047073
1,2,10014,C,CG,insertion,rs1558169263
2,2,10017,CACCC,C,indel,rs1558169385
3,2,10018,A,"AACA,AACG",insertion,rs1558169388
4,2,10019,CC,C,indel,rs1558169386
